### Spark HW2 Moive Recommendation
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

In [0]:
dbutils.library.installPyPI("koalas")

dbutils.library APIs are deprecated and will be removed in a future DBR release. You can use %pip commands to install notebook scoped python libraries.
Out[1]: True

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import databricks.koalas as ks

import warnings
warnings.filterwarnings("ignore")

In [0]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [0]:
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [0]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [0]:
ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [0]:
links_df.show(5)

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows



In [0]:
tags_df.show(5)

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
|     2|  89774|   Boxing story|1445715207|
|     2|  89774|            MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows



In [0]:
tmp0 = ratings_df.groupBy("userID").count()
tmp0.show(5)
#.select('count').rdd.min()[0]

+------+-----+
|userID|count|
+------+-----+
|   296|   27|
|   467|   22|
|   125|  360|
|   451|   34|
|     7|  152|
+------+-----+
only showing top 5 rows



In [0]:
tmp0 = ratings_df.groupBy("userID").count().select('count').rdd.min()
tmp0

Out[11]: Row(count=20)

In [0]:
tmp1 = ratings_df.groupBy("userID").count().select('count').rdd.min()[0]
tmp2 = ratings_df.groupBy("movieId").count().select('count').rdd.min()[0]
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1


In [0]:
tmp1 = ratings_df.groupBy("movieId").count().filter('count = 1').count()
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user


## Part 1: Spark SQL and OLAP

In [0]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### Q1: The number of Users

In [0]:
%sql 

Select count(distinct userID) as Number_of_users from ratings

Number_of_users
610


### Q2: The number of Movies

In [0]:
%sql 

Select count(distinct movieId) as Numer_of_movies from movies

Numer_of_movies
9742


### Q3:  How many movies are rated by users? List movies not rated before

In [0]:
%sql
Select m.title, m.genres from movies as m 
where m.movieId not in 
  (Select movieId from ratings)

title,genres
"Innocents, The (1961)",Drama|Horror|Thriller
Niagara (1953),Drama|Thriller
For All Mankind (1989),Documentary
"Color of Paradise, The (Rang-e khoda) (1999)",Drama
I Know Where I'm Going! (1945),Drama|Romance|War
"Chosen, The (1981)",Drama
"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance
Scrooge (1970),Drama|Fantasy|Musical
Proof (1991),Comedy|Drama|Romance
"Parallax View, The (1974)",Thriller


In [0]:
%sql 
Select count(distinct b.movieId) as Number_movies_are_rated_by_users from ratings as b

Number_movies_are_rated_by_users
9724


### Q4: List Movie Genres

In [0]:
%sql
Select distinct genres from movies

genres
Comedy|Horror|Thriller
Adventure|Sci-Fi|Thriller
Action|Adventure|Drama|Fantasy
Action|Drama|Horror
Action|Animation|Comedy|Sci-Fi
Animation|Children|Drama|Musical|Romance
Action|Adventure|Drama
Adventure|Sci-Fi
Documentary|Musical|IMAX
Adventure|Children|Fantasy|Sci-Fi|Thriller


In [0]:
%sql
Select distinct Category from movies
lateral view explode(split(genres,'[|]')) as Category order by Category

Category
(no genres listed)
Action
Adventure
Animation
Children
Comedy
Crime
Documentary
Drama
Fantasy


### Q5: Movie for Each Category

In [0]:
%sql
Select Category, count(movieId) as number from movies
lateral view explode(split(genres,'[|]')) as Category group by Category order by number desc

Category,number
Drama,4361
Comedy,3756
Thriller,1894
Action,1828
Romance,1596
Adventure,1263
Crime,1199
Sci-Fi,980
Horror,978
Fantasy,779


In [0]:
%sql
select t.Category, concat_ws(',',collect_set(t.title)) as list_of_movies from
  (
    Select Category, title from movies
    lateral view explode(split(genres,'[|]')) as Category 
    group by Category, title
    ) as t
group by t.Category

Category list_of_movies Crime Stealing Rembrandt (Rembrandt) (2003),The Gambler (2014),Girl with the Dragon Tattoo, The (Män som hatar kvinnor) (2009),Innocent Man, An (1989),Undertow (2004),Sorrow (2015),Shadow Dancer (2012),Sherlock Holmes and Dr. Watson: Acquaintance (1979),21 Jump Street (2012),Battles Without Honor & Humanity (Jingi naki tatakai) (1973),Blitz (2011),Alpha Dog (2007),Micmacs (Micmacs à tire-larigot) (2009),Alex Cross (2012),Perfect Crime, The (Crimen Ferpecto) (Ferpect Crime) (2004),Hellsinki (Rööperi) (2009),Be Cool (2005),Kingsman: The Secret Service (2015),Dark Passage (1947),Fast Five (Fast and the Furious 5, The) (2011),Desperate Living (1977),Net, The (1995),Analyze That (2002),Life Eternal (2015),High Sierra (1941),Run All Night (2015),Turner & Hooch (1989),Corruptor, The (1999),No Country for Old Men (2007),New Police Story (Xin jing cha gu shi) (2004),Pledge, The (2001),Untraceable (2008),Boy A (2007),Carlito's Way (1993),Jane Austen's Mafia! (1998),Hobo with a Shotgun (2011),Die Hard (1988),Dead Man Walking (1995),Cobra (1986),Saw IV (2007),Man on a Ledge (2012),Thief of Paris, The (Le voleur) (1967),The Falcon and the Snowman (1985),Brick Mansions (2014),Boiler Room (2000),Furious 7 (2015),Batman Forever (1995),Short Film About Killing, A (Krótki film o zabijaniu) (1988),Flashback (1990),Unleashed (Danny the Dog) (2005),Crank: High Voltage (2009),Femme Nikita, La (Nikita) (1990),Cellular (2004),Sexy Beast (2000),Player, The (1992),Man Who Wasn't There, The (2001),Three Burials of Melquiades Estrada, The (2006),Angels with Dirty Faces (1938),Flickering Lights (Blinkende lygter) (2000),Papillon (1973),Band of Outsiders (Bande à part) (1964),Good Time (2017),Hamlet (1996),Himizu (2011),Godfather: Part II, The (1974),New Jersey Drive (1995),Herod's Law (Ley de Herodes, La) (2000),Our Lady of the Assassins (Virgen de los sicarios, La) (2000),One False Move (1992),Dark Portals: The Chronicles of Vidocq (Vidocq) (2001),F/X2 (a.k.a. F/X 2 - The Deadly Art of Illusion) (1991),Lawless (2012),Fresh (1994),Wild Things (1998),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Tower Heist (2011),General, The (1998),Another Thin Man (1939),Money Train (1995),Scarface (1932),Faster Pussycat! Kill! Kill! (1965),All About the Benjamins (2002),Prime Suspect 2 (1992),Investigation of a Citizen Above Suspicion (Indagine su un cittadino al di sopra di ogni sospetto) (1970),The Nice Guys (2016),Max Payne (2008),Adventures Of Sherlock Holmes And Dr. Watson: The Twentieth Century Approaches (1986),Police Academy 5: Assignment: Miami Beach (1988),Harry Brown (2009),Sherlock Holmes: Dressed to Kill (1946),Extreme Ops (2002),M (1931),Best Men (1997),Leaves of Grass (2009),Big Deal on Madonna Street (I Soliti Ignoti) (1958),House of Games (1987),Nine to Five (a.k.a. 9 to 5) (1980),Small Time Crooks (2000),Set It Off (1996),Ninja (2009),Tsotsi (2005),Suddenly (1954),Real McCoy, The (1993),Henry: Portrait of a Serial Killer (1986),Hangover, The (2009),Scarface (1983),Fire with Fire (2012),Postman Always Rings Twice, The (1946),Lethal Weapon (1987),Bus 174 (Ônibus 174) (2002),Seven-Per-Cent Solution, The (1976),Scorched (2003),You Only Live Once (1937),Get the Gringo (2012),Tokyo Tribe (2014),Office Space (1999),Shepherd: Border Patrol, The (2008),Grand Canyon (1991),Double, The (2011),By the Gun (2014),I Am a Fugitive from a Chain Gang (1932),Clear and Present Danger (1994),Munich (2005),Brothers Bloom, The (2008),Ocean's Eleven (a.k.a. Ocean's 11) (1960),Gone in 60 Seconds (2000),Trailer Park Boys (1999),Lord of War (2005),Lethal Weapon 3 (1992),Limey, The (1999),Yellow Sea, The (a.k.a. The Murderer) (Hwanghae) (2010),Into the Blue (2005),Death of a President (2006),Killing Them Softly (2012),City of God (Cidade de Deus) (2002),Mobsters (1991),Midnight in the Garden of Good and Evil (1997),Youth of the Beast (Yaju no seishun) (1963),Time Lapse (2014),Incognito (1997),Heartbreakers (2001),Baby Boy (2001),Gridlock'd (1997),R

## Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [0]:
ratings_df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [0]:
movie_ratings=ratings_df.drop('timestamp')

In [0]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [0]:
movie_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [0]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [0]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.8,0.2], seed = 2020)

In [0]:
#Create ALS model
als = ALS(maxIter=5, rank=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [0]:
#Tune model using ParamGridBuilder
paramGrid = ParamGridBuilder()\
            .addGrid(als.regParam, [0.1, 0.01, 0.001])\
            .addGrid(als.maxIter, [3, 5, 10])\
            .addGrid(als.rank, [5, 10, 15])\
            .build()

In [0]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [0]:
# Build Cross validation 
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

In [0]:
#Fit ALS model to training data
model = als.fit(training)

In [0]:
#Extract best model from the tuning exercise using ParamGridBuilder
cvModel = crossval.fit(training)
predictions = cvModel.transform(training)
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.6366976701173241


### Model testing
And finally, make a prediction and check the testing error.

In [0]:
#Generate predictions and evaluate using RMSE
best_model = cvModel.bestModel
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [0]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank:"+str(best_model._java_obj.parent().getRank())), 
print (" MaxIter:"+str(best_model._java_obj.parent().getMaxIter())), 
print (" RegParam:"+str(best_model._java_obj.parent().getRegParam()))

RMSE = 0.8825373513583171
**Best Model**
 Rank:5
 MaxIter:10
 RegParam:0.1


In [0]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|   1197|   3.0| 3.5554774|
|   148|   5816|   4.0| 3.5437984|
|   148|  40629|   5.0| 3.2765374|
|   148|  76093|   3.0|  3.861536|
|   148| 112852|   3.5| 3.6920197|
|   148| 134853|   4.0| 3.7789242|
|   148| 157296|   3.0|  3.274326|
|   148| 160718|   4.5| 3.8184693|
|   463|   5952|   5.0| 3.9378443|
|   463|   6539|   3.5| 3.9688256|
|   471|    527|   4.5| 3.8805285|
|   471|   2324|   5.0| 3.9920485|
|   471|   6377|   4.0| 3.5673406|
|   471|   7147|   4.0|  3.572465|
|   471|   8636|   2.5| 3.1978924|
|   471|  92259|   4.5| 3.8563082|
|   471| 158966|   4.5|  4.537438|
|   496|  40826|   4.0| 3.0748262|
|   496|  99149|   4.0| 3.2979934|
|   243|     62|   5.0| 4.3324656|
+------+-------+------+----------+
only showing top 20 rows



### Model apply and see the performance

In [0]:
alldata=best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6911646578147499


In [0]:
alldata.registerTempTable("alldata")

In [0]:
%sql 
select * from alldata

userId,movieId,rating,prediction
148,356,4.0,3.6099083
148,1197,3.0,3.5554774
148,4308,4.0,3.566312
148,4886,3.0,3.6114333
148,4896,4.0,3.5139458
148,4993,3.0,3.395151
148,5618,3.0,3.5087624
148,5816,4.0,3.5437984
148,5952,3.0,3.4494274
148,6377,3.0,3.7033498


In [0]:
%sql 
select * from movies join alldata on movies.movieId=alldata.movieId

movieId,title,genres,userId,movieId,rating,prediction
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,4.0,4.755186
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,1,4.0,3.6935697
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,1,4.5,3.8008304
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,1,2.5,3.310832
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,1,4.5,4.016884
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,18,1,3.5,3.92744
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,1,4.0,3.2239318
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,21,1,3.5,3.382154
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,27,1,3.0,4.03089
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,31,1,5.0,4.27227


## Recommend moive to users with id: 575, 232. 
you can choose some users to recommend the moives

In [0]:
userRecs = best_model.recommendForAllUsers(10)

In [0]:
display(userRecs.filter(userRecs.userId == 575))

userId,recommendations
575,"List(List(8477, 6.1207714), List(40491, 5.897125), List(6818, 5.836458), List(84847, 5.753885), List(3266, 5.715936), List(148881, 5.657381), List(53123, 5.641), List(59018, 5.623018), List(60943, 5.623018), List(74754, 5.5585394))"


In [0]:
user_recommendation = userRecs.to_koalas()

In [0]:
user_recommendation.head()

,userId,recommendations
0,1,"[(33649, 6.1121416091918945), (3379, 6.0713763..."
1,3,"[(8138, 5.850666046142578), (132800, 5.3251514..."
2,5,"[(25825, 5.102529525756836), (720, 4.970302104..."
3,6,"[(3567, 5.117492198944092), (1046, 5.111182689..."
4,9,"[(8477, 5.762623310089111), (40491, 5.56524419..."


In [0]:
movies_koalas = movies_df.to_koalas()

In [0]:
def movie_recommendation(user_recommendation, userId, movies_koalas):
  rec_movieId = []
  for item in user_recommendation.loc['userId' == userId][1]:
    rec_movieId.append(item[0])
  return movies_koalas.loc[movies_koalas.movieId.isin(rec_movieId)]

In [0]:
movie_recommendation(user_recommendation, 575, movies_koalas)

,movieId,title,genres
2523,3379,On the Beach (1959),Drama
3320,4495,Crossing Delancey (1988),Comedy|Romance
4251,6201,Lady Jane (1986),Drama|Romance
5037,7842,Dune (2000),Drama|Fantasy|Sci-Fi
5848,32582,"Wild Parrots of Telegraph Hill, The (2003)",Documentary
5906,33649,Saving Face (2004),Comedy|Drama|Romance
7177,72171,Black Dynamite (2009),Action|Comedy
7521,84273,Zeitgeist: Moving Forward (2011),Documentary
8591,117531,Watermark (2014),Documentary
9514,171495,Cosmos,(no genres listed)


In [0]:
movie_recommendation(user_recommendation, 232, movies_koalas)

,movieId,title,genres
2523,3379,On the Beach (1959),Drama
3320,4495,Crossing Delancey (1988),Comedy|Romance
4251,6201,Lady Jane (1986),Drama|Romance
5037,7842,Dune (2000),Drama|Fantasy|Sci-Fi
5848,32582,"Wild Parrots of Telegraph Hill, The (2003)",Documentary
5906,33649,Saving Face (2004),Comedy|Drama|Romance
7177,72171,Black Dynamite (2009),Action|Comedy
7521,84273,Zeitgeist: Moving Forward (2011),Documentary
8591,117531,Watermark (2014),Documentary
9514,171495,Cosmos,(no genres listed)


## Find the similar moives for moive with id: 463, 471
You can find the similar moives based on the ALS results

In [0]:
item_factors = best_model.itemFactors

In [0]:
movie_factors = item_factors.to_koalas()

In [0]:
movie_factors.head()

,id,features
0,10,"[1.4937762, 1.3586565, 0.28697157, -0.08016688..."
1,20,"[1.1228839, 1.1236956, -0.14102137, -0.4190420..."
2,30,"[0.62362033, 0.4254547, -0.17052107, -1.868791..."
3,40,"[2.007661, 1.0175308, 0.38295424, -0.13591366,..."
4,50,"[1.6928099, 1.5246452, 0.34827, -0.8071215, -0..."


In [0]:
movie_factors['features']

Out[43]: 0      [1.4937762, 1.3586565, 0.28697157, -0.08016688...
1      [1.1228839, 1.1236956, -0.14102137, -0.4190420...
2      [0.62362033, 0.4254547, -0.17052107, -1.868791...
3      [2.007661, 1.0175308, 0.38295424, -0.13591366,...
4      [1.6928099, 1.5246452, 0.34827, -0.8071215, -0...
5      [1.6419759, 0.76810795, 0.23333535, 0.39288637...
6      [1.5541062, 1.1542326, 0.2925609, -0.7459551, ...
7      [1.4355348, 1.094812, 0.61560625, -0.89683026,...
8      [1.1847144, 0.83618253, 0.4086571, 0.004296299...
9      [1.734194, 1.542322, 0.12119383, -0.24628009, ...
10     [1.3553396, 1.1002647, 0.80070657, -0.06266281...
11     [1.4667892, 1.6329945, 0.035447057, -0.0261774...
12     [1.2353214, 1.0347548, 0.37435505, 0.65154606,...
13     [1.6314442, 0.84576637, 0.688034, 0.07593372, ...
14     [1.4262197, 1.2742668, 0.046442073, -0.8816381...
15     [0.5947727, 0.61685455, 0.55393124, -1.7499311...
16     [1.4755222, 0.88913625, 0.2818376, 0.38379213,...
17     [1.3916231, 0.6

In [0]:
def similar_movies(features, movieId):

  try: 
    target_id_feature = movie_factors.loc[movie_factors.id == movieId].features.to_numpy()[0]
  except:
    return 'There is no movie with id ' + str(movieId)

  similarities = []
  for feature in movie_factors['features'].to_numpy():
    similarity = np.dot(target_id_feature,feature)/(np.linalg.norm(target_id_feature) * np.linalg.norm(feature))
    similarities.append(similarity)
    
  ks_similarity = ks.DataFrame({'similarity' : similarities}, index = movie_factors.id.to_numpy())
  # top 11 similar movies contain the movie itself with similarity = 1, so I need to remove it. 
  top_11 = ks_similarity.sort_values(by = ['similarity'], ascending = False).head(11)
  joint = top_11.merge(movies_koalas, left_index=True, right_on = 'movieId', how = 'inner')
  joint.sort_values(by = ['similarity'], ascending = False,inplace = True)
  joint.reset_index(inplace = True)
  # take top 10 similar movies
  return joint.loc[1:,['movieId','title','genres']]

In [0]:
similar_movies(features = movie_factors['features'], movieId = 463)

Out[45]: 'There is no movie with id 463'

In [0]:
similar_movies(features = movie_factors['features'], movieId = 471)

,movieId,title,genres
1,2956,Someone to Watch Over Me (1987),Action|Crime|Thriller
2,2583,Cookie's Fortune (1999),Comedy|Drama
3,111443,Chef (2014),Comedy
4,4587,Earth Girls Are Easy (1988),Comedy|Musical|Sci-Fi
5,5970,My Girl (1991),Comedy|Drama|Romance
6,1033,"Fox and the Hound, The (1981)",Animation|Children|Drama
7,36,Dead Man Walking (1995),Crime|Drama
8,1446,Kolya (Kolja) (1996),Comedy|Drama
9,3478,"Bamba, La (1987)",Drama
10,3097,"Shop Around the Corner, The (1940)",Comedy|Drama|Romance


In [0]:
similar_movies(features = movie_factors['features'], movieId = 500)

,movieId,title,genres
1,587,Ghost (1990),Comedy|Drama|Fantasy|Romance|Thriller
2,413,Airheads (1994),Comedy
3,3988,How the Grinch Stole Christmas (a.k.a. The Gri...,Children|Comedy|Fantasy
4,2310,"Mighty, The (1998)",Drama
5,2085,101 Dalmatians (One Hundred and One Dalmatians...,Adventure|Animation|Children
6,6890,Elephant (2003),Drama
7,59143,Super High Me (2007),Comedy|Documentary
8,4090,"Brave Little Toaster, The (1987)",Animation|Children
9,8614,Overboard (1987),Comedy|Romance
10,586,Home Alone (1990),Children|Comedy


In [0]:
similar_movies(features = movie_factors['features'], movieId = 101)

,movieId,title,genres
1,8937,Friday Night Lights (2004),Action|Drama
2,3432,Death Wish 3 (1985),Action|Drama
3,1256,Duck Soup (1933),Comedy|Musical|War
4,27879,DiG! (2004),Documentary
5,8370,"Blind Swordsman: Zatoichi, The (Zatôichi) (2003)",Action|Comedy|Crime|Drama
6,3148,"Cider House Rules, The (1999)",Drama
7,1277,Cyrano de Bergerac (1990),Comedy|Drama|Romance
8,1365,Ridicule (1996),Drama
9,4641,Ghost World (2001),Comedy|Drama
10,735,Cemetery Man (Dellamorte Dellamore) (1994),Horror


## Write the report 
During my daily life, I'm not quite satisfied with many recommended music and movies provided by some apps and websites, even those from Big Names like Youtube and Netflix. As a data scientist candidate, I would like to improve those recommendation systems in the future if I have a chance. Before that, I need to have a general and basic idea about how the most common and popular recommendation systems work. So I tried my best to build a simple recommendation system on a small dataset. 
<br>
1. Conduct exploratory data analysis like split genres into categories, count the number of movies under each category, etc.
2. Build a recommendation model based on historical movie ratings and solve it by matrix factorization and alternating least squares (ALS). 
3. Tune the model parameters through grid search and crossvalidation with the metric root mean square errors (rmse). The optimal model has rmse = 0.64 on the training dataset and rmse = 0.889 on the test dataset.
4. Recommend 10 movies to some certain users and find top 10 similar movies in terms of some specific movies.

The best model I got has 5 latent factors (hidden features). These features describe a movie in 5 dimensions. Based on the feature, I can define the similarity between movies.<br>
In this model, I droped all unseen items and/or items during training process. The evaluation metric was computed over the non-NaN data and so, was valid.
<br>
I tried twice to run it on the large dataset for the whole night. Maybe due to the community version, everytime when training the model more than 2 hours, my Spark broke down.
In the future, I will run this model on a much larger dataset.